In [1]:
import findspark
findspark.init()
import pyspark
sc = pyspark.SparkContext()

from pyspark.sql.types import *
from pyspark.sql import SparkSession

spark = SparkSession.builder \
                    .appName('Test face') \
                    .getOrCreate()

In [2]:
userdata = spark.read.parquet('C:\\Users\\mroberts\\Documents\\userdata1.parquet')

In [3]:
userdata.dtypes

[('registration_dttm', 'timestamp'),
 ('id', 'int'),
 ('first_name', 'string'),
 ('last_name', 'string'),
 ('email', 'string'),
 ('gender', 'string'),
 ('ip_address', 'string'),
 ('cc', 'string'),
 ('country', 'string'),
 ('birthdate', 'string'),
 ('salary', 'double'),
 ('title', 'string'),
 ('comments', 'string')]

In [17]:
userdata.count()

1000

In [5]:
userdata.schema

StructType(List(StructField(registration_dttm,TimestampType,true),StructField(id,IntegerType,true),StructField(first_name,StringType,true),StructField(last_name,StringType,true),StructField(email,StringType,true),StructField(gender,StringType,true),StructField(ip_address,StringType,true),StructField(cc,StringType,true),StructField(country,StringType,true),StructField(birthdate,StringType,true),StructField(salary,DoubleType,true),StructField(title,StringType,true),StructField(comments,StringType,true)))

In [8]:
userdata[['gender','salary']].describe().show()

+-------+------+------------------+
|summary|gender|            salary|
+-------+------+------------------+
|  count|  1000|               932|
|   mean|  null|149005.35665236053|
| stddev|  null| 79785.17673591433|
|    min|      |          12380.49|
|    max|  Male|         286592.99|
+-------+------+------------------+



In [9]:
userdata.explain()

== Physical Plan ==
*FileScan parquet [registration_dttm#0,id#1,first_name#2,last_name#3,email#4,gender#5,ip_address#6,cc#7,country#8,birthdate#9,salary#10,title#11,comments#12] Batched: true, Format: Parquet, Location: InMemoryFileIndex[file:/C:/Users/mroberts/Documents/userdata1.parquet], PartitionFilters: [], PushedFilters: [], ReadSchema: struct<registration_dttm:timestamp,id:int,first_name:string,last_name:string,email:string,gender:...


In [10]:
userdata.createTempView('userd')

In [13]:
gender_salary = spark.sql("select gender, title, avg(salary) as avg_salary from userd group by gender, title order by title, gender")

In [14]:
gender_salary.write.save('C:\\Users\\mroberts\\Documents\\genderData.parquet')

In [15]:
test = spark.read.parquet('C:\\Users\\mroberts\\Documents\\genderData.parquet')

In [16]:
test.show()

+------+--------------------+------------------+
|gender|               title|        avg_salary|
+------+--------------------+------------------+
|Female|Business Systems ...|148044.91666666666|
|  Male|Business Systems ...|         122541.72|
|Female|Payment Adjustmen...|        207025.212|
|  Male|Payment Adjustmen...|186480.70666666667|
|Female|Computer Systems ...|          68729.56|
|  Male|Computer Systems ...| 93216.20000000001|
|Female|Information Syste...|         199784.64|
|  Male|Information Syste...| 87605.82500000001|
|Female|Software Test Eng...|         214031.75|
|Female|Software Test Eng...|           92837.5|
|  Male|Computer Systems ...|168114.96000000002|
|  Male|Computer Systems ...|         179115.49|
|Female|Computer Systems ...|          90859.41|
|  Male|Computer Systems ...|          99291.05|
|Female|Physical Therapy ...|          138081.4|
|  Male|Physical Therapy ...|171580.77833333332|
|Female|     Statistician IV|          95828.75|
|Female|Structural A

Extra info on interacting with hive:
https://spark.apache.org/docs/latest/sql-programming-guide.html#hive-tables
